In [1]:
# Importing necessary Libraries

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Importing Dataset

datafile = 'gdrive/My Drive/ResearchPaper/parkinson/pd_speech_features.csv'
df = pd.read_csv(datafile, header=1)

In [3]:
df.head()

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,GNE_SNR_TKEO,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,0.26512,0.083127,...,0.071728,0.010352,-2.73030,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,0.22004,0.127410,...,0.729330,0.780410,5.22940,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,0.15756,0.116890,...,0.269510,-0.005522,0.35054,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,0.17295,0.147370,...,0.366920,-0.492650,0.19164,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.3

In [4]:
# Dataset Description

print('Given Features: ', df.columns)
print('---------------------------------------------------------')
print('Dimensions: ', df.shape)

Given Features:  Index(['id', 'gender', 'PPE', 'DFA', 'RPDE', 'numPulses', 'numPeriodsPulses',
       'meanPeriodPulses', 'stdDevPeriodPulses', 'locPctJitter',
       ...
       'tqwt_kurtosisValue_dec_28', 'tqwt_kurtosisValue_dec_29',
       'tqwt_kurtosisValue_dec_30', 'tqwt_kurtosisValue_dec_31',
       'tqwt_kurtosisValue_dec_32', 'tqwt_kurtosisValue_dec_33',
       'tqwt_kurtosisValue_dec_34', 'tqwt_kurtosisValue_dec_35',
       'tqwt_kurtosisValue_dec_36', 'class'],
      dtype='object', length=755)
---------------------------------------------------------
Dimensions:  (756, 755)


In [5]:
print(df.describe())

               id      gender  ...  tqwt_kurtosisValue_dec_36       class
count  756.000000  756.000000  ...                 756.000000  756.000000
mean   125.500000    0.515873  ...                  31.481110    0.746032
std     72.793721    0.500079  ...                  34.230991    0.435568
min      0.000000    0.000000  ...                   2.364000    0.000000
25%     62.750000    0.000000  ...                   3.948750    0.000000
50%    125.500000    1.000000  ...                  10.637250    1.000000
75%    188.250000    1.000000  ...                  61.125325    1.000000
max    251.000000    1.000000  ...                 156.423700    1.000000

[8 rows x 755 columns]


In [0]:
# Separating features and results

X = df.drop(['class'], axis=1)
y = df['class']

In [7]:
print('Dimensions of X: ', X.shape)
print('Dimensions of y: ', y.shape)

Dimensions of X:  (756, 754)
Dimensions of y:  (756,)


In [0]:
# Importing Classification models

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [0]:
# Function for Models

def modelfunc(X_train, X_test, y_train, y_test, ovr, acc, models):
  
  i = 0
  
  for model in models:
    
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    testacc = accuracy_score(y_test, pred)*100
    pred_train = model.predict(X_train)
    trainacc = accuracy_score(y_train, pred_train)*100
    
    if ovr[i] != min(ovr[i], abs(testacc - trainacc)):
      
      ovr[i] = abs(testacc - trainacc)
      acc[i] = testacc
      
    
    i = i + 1 
    
  return acc

In [0]:
from sklearn.model_selection import KFold

In [11]:
kf = KFold(n_splits=10, random_state=101, shuffle=True)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=101, shuffle=True)


In [0]:
  lr = LogisticRegression(random_state=101)
  
  dtc = DecisionTreeClassifier(random_state=101)
  
  svc = SVC(gamma='auto', random_state=101)
  
  gnb = GaussianNB()
  
  rfc = RandomForestClassifier(n_estimators=10, n_jobs=-1, random_state=101, max_depth=2)
  
  knc = KNeighborsClassifier(n_neighbors=7, n_jobs=-1, algorithm='auto')
  
  km = KMeans(n_clusters=2, n_jobs=-1, random_state=101)
  
  gb = GradientBoostingClassifier(n_estimators=100, learning_rate= 0.01, max_depth=12, random_state=101)
  
  models = [lr, dtc, svc, gnb, rfc, knc, km, gb]

In [13]:
ovr = np.asarray([100, 100, 100, 100, 100, 100, 100, 100])
acc = np.zeros(len(models))

for train_idx, test_idx in kf.split(X):
 # print("TRAIN: ", train_idx, "TEST: ", test_idx)
  X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
  y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
  
  acc = modelfunc(X_train, X_test, y_train, y_test, ovr, acc, models)
  

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [14]:
acc

array([77.63157895, 84.21052632, 82.89473684, 76.31578947, 84.21052632,
       80.26315789, 26.66666667, 85.52631579])

           DecisionTree     SVC        GaussianNB   RandomForest  KNeighbors       

**PCA**

In [0]:
from sklearn.decomposition import PCA as sklearnPCA

In [0]:
pca = sklearnPCA(n_components=25)

In [0]:
principalcomponents = pca.fit_transform(X)

In [23]:
principalcomponents

array([[-5.81581617e+09,  4.70139059e+06, -6.21026434e+05, ...,
         8.01212744e+03,  2.28769131e+03,  2.06088695e+04],
       [-6.02939880e+09,  4.61012160e+06, -4.19808109e+05, ...,
         1.22648334e+04,  1.02576272e+04, -3.45731165e+03],
       [-6.15918366e+09,  5.57872047e+06, -4.11510760e+05, ...,
         1.56497982e+04,  9.98636057e+03,  2.57372276e+04],
       ...,
       [ 2.21968907e+09, -1.17424165e+07, -1.66780957e+06, ...,
        -4.53489923e+04,  7.58248327e+03,  1.74861970e+04],
       [-7.65754098e+08, -1.99305252e+06, -2.45006049e+06, ...,
        -9.17126752e+03, -1.59314552e+03, -2.03295468e+04],
       [-9.47917456e+08,  1.29502932e+06, -2.83121286e+06, ...,
        -2.42687594e+04, -1.21487551e+03, -7.05979038e+04]])

In [0]:
principalDF = pd.DataFrame(data = principalcomponents)

In [25]:
principalDF

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24
0,-5.815816e+09,4.701391e+06,-6.210264e+05,-5.381659e+05,-2.002537e+06,5.689575e+05,3.121118e+05,-1.129303e+05,-1.869113e+04,1.446696e+05,4.332739e+05,1.114193e+05,-9.398459e+02,-3.574996e+04,1.177001e+05,150613.417373,-9.264008e+04,2.949682e+04,2.768452e+04,-10068.503407,10879.763395,98703.714661,8012.127440,2287.691313,20608.869451
1,-6.029399e+09,4.610122e+06,-4.198081e+05,-5.848704e+05,-2.241594e+06,6.585137e+05,2.439150e+05,-1.469889e+05,-3.255583e+04,1.449871e+05,4.352602e+05,1.331296e+05,-1.067872e+04,-5.698373e+04,9.849697e+04,49685.378300,-6.689129e+04,2.515385e+04,1.151154e+04,-64129.128157,11135.081223,88945.739691,12264.833383,10257.627239,-3457.311648
2,-6.159184e+09,5.578720e+06,-4.115108e+05,-5.529822e+05,-2.333541e+06,6.770039e+05,2.835127e+05,-1.458654e+05,-6.718523e+04,1.391673e+05,4.647172e+05,1.254765e+05,5.890738e+03,-5.077598e+04,9.157826e+04,66246.256665,-8.034465e+04,3.272135e+04,2.084932e+04,-90699.082898,13583.220509,91021.115097,15649.798210,9986.360568,25737.227567
3,-7.197145e+09,5.943199e+07,1.355807e+06,5.389349e+06,4.936082e+05,2.910622e+06,4.668452e+06,3.077677e+06,-4.545529e+05,5.080480e+05,-6.534829e+05,1.971747e+05,1.733208e+05,1.905147e+04,-2.558746e+05,-158438.800205,3.067944e+05,7.441011e+05,1.149156e+05,-84939.205700,103724.112672,22633.137385,-182043.976492,47530.173969,28844.824278
4,-6.423357e+09,8.714129e+07,1.791626e+07,8.141098e+06,-1.397339e+06,-3.986382e+06,-1.417338e+06,3.502087e+06,-1.469398e+06,-3.083386e+06,-1.046234e+06,8.289180e+05,5.596608e+05,-1.719060e+06,7.314277e+05,-522554.039397,7.661186e+05,-7.873383e+05,-1.386010e+05,-48252.995683,-316281.879608,-20435.767937,-197312.394286,52695.107682,11279.789610
5,-3.818902e+09,3.152648e+07,2.169077e+06,-3.809258e+06,2.592769e+06,3.817376e+06,1.567148e+06,2.311891e+06,1.328136e+06,-1.876569e+06,-1.132673e+06,1.872565e+06,-2.778205e+06,-9.551469e+04,-6.429005e+05,270607.545282,-1.200138e+05,-1.361080e+06,-3.274848e+05,-50260.227364,-245345.420214,41380.253165,192020.114322,-42065.675331,-47161.749243
6,-1.490717e+09,-6.762254e+06,-1.715183e+06,-3.284816e+05,4.426022e+05,-2.457879e+05,6.081969e+04,-9.272264e+04,1.515625e+05,1.566601e+05,-4.041847e+05,-1.937736e+05,-7.546786e+04,1.124602e+05,5.284970e+04,248740.791694,-6.171548e+04,-3.457119e+04,-5.229858e+03,127712.395866,-3432.881117,-9642.007326,-32754.779343,5859.624626,28124.683188
7,-2.081315e+09,-2.363895e+05,-2.406266e+06,1.868930e+04,6.968464e+05,-1.469971e+05,2.746730e+05,-8.881817e+04,-8.019376e+04,1.461748e+05,-2.998543e+05,-1.470787e+05,-6.742486e+04,8.900531e+04,3.093669e+04,163103.141562,-4.122900e+04,-2.133483e+04,-2.300927e+04,33787.299547,4196.235354,63386.430426,-39196.617947,12489.338049,30912.599255
8,-1.994097e+09,-2.776081e+05,-2.236301e+06,-7.560882e+05,6.803806e+05,-2.025180e+05,-6.034995e+04,-1.473607e+05,-2.239004e+05,-3.310865e+04,-1.818597e+05,-1.435377e+05,-2.209894e+04,2.841612e+04,1.121766e+05,129726.124445,-4.683490e+04,-3.085827e+04,-1.847381e+04,81161.702804,-1327.980719,65883.911483,-8695.224635,-3536.753600,3633.894246
9,9.967964e+09,8.161473e+06,-3.572014e+06,-5.468584e+05,1.651212e+06,-1.113884e+06,2.484943e+04,-9.012028e+04,-3.181627e+05,-2.075893e+05,3.008001e+05,-3.862516e+04,8.517041e+04,-4.812389e+04,1.666885e+05,308943.832990,-8.839416e+04,5.033841e+04,4.097547e+04,10335.050074,2187.740903,25778.439785,41002.723662,-17642.193937,-55279.451687
